# Importações

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time



# Função para coletar dados conforme número de páginas e retornar um dataframe

In [3]:
def get_imoveis(num_paginas):
    #definição do dataframe principal
    df_all = pd.DataFrame(columns=['Zona','Preco', 'Metragem','Quartos'])    
    
    #definição das regiões dos dados a serem extraidos
    zonas =['zona-norte','zona-leste','zona-oeste','centro','zona-sul']
    
    for zona in zonas:
        url = 'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-'+zona+'/'       
        num_pag=1
        preco_list=[]
        metragem_list=[]
        quartos_list=[]
        #Extração de cada pagina 
        while num_pag <=num_paginas:
            if num_pag != 1:
               #cada pagina do ML tem 48 anuncions
               num_anuncios=1+48*(num_pag-1)
               rp=requests.get(url+'_Desde_'+ str(num_anuncios)).content
            else:
               rp=requests.get(url).content
            time.sleep(2)
            soup=BeautifulSoup(rp)
            precos = soup.find_all("span", {"class":"price-tag-fraction"})
            attribs = soup.find_all("ul", {"class":"ui-search-card-attributes ui-search-item__group__element"})
            #Transformacao dos precos e atributos
            for preco in precos:
                preco_list.append(str(preco.text).replace('.',''))
            for attrib in attribs: 
                metragem_list.append(attrib.text.split()[0])
                quartos_list.append(attrib.text.split()[2].replace('construídos',''))
            num_pag=num_pag+1
            #criacao de um dataframe para as regiões
            df_zona = pd.DataFrame(zip(preco_list,metragem_list,quartos_list) , columns = ['Preco', 'Metragem','Quartos'])
            df_zona['Zona'] = pd.Series([zona for x in range(len(df_zona.index))])
        df_all=df_all.append(df_zona,ignore_index=True)
    return df_all     

In [4]:
df_zonas=get_imoveis(5)
df_zonas.info()
df_zonas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Zona      1200 non-null   object
 1   Preco     1200 non-null   object
 2   Metragem  1200 non-null   object
 3   Quartos   1200 non-null   object
dtypes: object(4)
memory usage: 37.6+ KB


,Zona,Preco,Metragem,Quartos
0,zona-norte,1400,67,2
1,zona-norte,1000,60,2
2,zona-norte,1220,51,2
3,zona-norte,2000,87,2
4,zona-norte,1700,61,2
...,...,...,...,...
1195,zona-sul,2700,26,1
1196,zona-sul,1350,64,2
1197,zona-sul,4500,168,2
1198,zona-sul,2200,200,4
